#### Import statements

In [1]:
import torch
import os
import shutil

#### Load the drive folder containing all required files

In [2]:
# mount the drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# access the drive folder containing everything we need
%cd /content/drive/My Drive/Colab environments/Risiko! DL

# check that we are in the desired folder
%ls

Mounted at /content/drive
/content/drive/My Drive/Colab environments/Risiko! DL
 3D_models/                                'Risiko! Test.ipynb'
 backgrounds/                               runs/
 coco_risiko.yaml                           synthetic_dataset/
 custom_complete_yolo.yaml                  synthetic_images/
 custom_yolo.yaml                           tanks_flags_detection.ipynb
 datasets/                                  test_example.txt
 pre_trained_weights/                       test.txt
 real_images/                               yolov5/
'Risiko! Synthetic Dataset Creator.ipynb'


#### Clone the GitHub repository yolov5 and install requirements

In [3]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/drive/My Drive/Colab environments/Risiko! DL/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.0/613.0 kB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.1 MB/s eta 0:00:00


#### Model definition

In [4]:
# number of classes: 6 tanks + 6 flags = 12 total classes
NUM_CLASSES = 12

# model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, autoshape=False, force_reload=True, classes=NUM_CLASSES)

# set the device
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# save the model to the desired device
model.to(device)


/usr/local/lib/python3.10/dist-packages/torch/hub.py:286: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2023-7-5 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)

Overriding model.yaml nc=80 with nc=12

                 from  n    params  module                                  arguments                     
  0                -1 

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3(
      (cv1): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
     

#### Move the $.yaml$ file inside the $yolov5$ folder

In [6]:
# current path is $yolov5 folder
current_path = os.getcwd()
# name of the file containing training specifications
filename = "custom_complete_yolo.yaml"
# source path of the file
file_path = os.path.join(current_path, "..", filename)
# path to the folder where the file has to be moved
dest_path = os.path.join(current_path, filename)
# copy the file in $file_path to $dest_path
shutil.copy(file_path, dest_path)

'/content/drive/MyDrive/Colab environments/Risiko! DL/yolov5/custom_complete_yolo.yaml'

#### Fine-tuning the model

In [7]:
!python train.py --img 640 --epochs 300 --batch-size 32 --cache RAM --data custom_complete_yolo.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=custom_complete_yolo.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=32, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=RAM, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 82 (delta 42), reused 53 (delta 37), pack-reused 9
Unpacking objects: 100% (82/82), 151.29 KiB | 113.00 KiB/s, done.
Command 'git fetch origin' timed out after 5 seconds
fatal: cannot change to '/content/drive/MyDrive/Colab': No such file or directory